# 01 Import libraries

In [1]:
#import libraries (p1,q3)
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# 02 Import Data

In [2]:
# create path (p1,q3)
path = r'C:\Users\erhil\OneDrive\Career Foundry Program\Immersion Data Analytics Course\A4 - Python Fundamentals for Data Analysts\02-2022 Instacart Basket Analysis'

In [3]:
#import data (p1,q3)
df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)
df_opm = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_4.pkl'))


# 03 Explore & Wrangle customer dataset

In [4]:
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
df_cust.columns

Index(['user_id', 'First Name', 'Surnam', 'Gender', 'STATE', 'Age',
       'date_joined', 'n_dependants', 'fam_status', 'income'],
      dtype='object')

In [6]:
df_cust['date_joined'].value_counts(dropna = False)

9/17/2018     213
2/10/2018     212
4/1/2019      211
9/21/2019     211
12/19/2017    210
             ... 
9/1/2018      141
1/22/2018     140
11/24/2017    139
7/18/2019     138
8/6/2018      128
Name: date_joined, Length: 1187, dtype: int64

### rename column 'Surnam' to 'Last Name'

In [7]:
#rename 'Surnam' to 'LastName' (p1,q4)
df_cust.rename(columns = {'Surnam':'Last Name'}, inplace = True)

In [8]:
#did not drop any columns yet since I do not see any that add no value
#also do not know what I'll need to analyze, so left all variables present

### data quality checks

In [9]:
#(p1,q5) 
df_cust.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [10]:
#check for null values in data set
df_cust.isnull().sum()

user_id             0
First Name      11259
Last Name           0
Gender              0
STATE               0
Age                 0
date_joined         0
n_dependants        0
fam_status          0
income              0
dtype: int64

In [15]:
# split df into null and no null First Names
df_no_nulls = df_cust[df_cust['First Name'].isnull()==False]
df_w_nulls = df_cust[df_cust['First Name']. isnull()==True]

In [16]:
df_w_nulls.head()

,user_id,First Name,Last Name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819


In [17]:
# There are 11,259 null values in the 'First Name' column.
# This does not influence any values in the analysis, so I'll leave them as NaN.

In [18]:
#create a df containing only duplicated data in df_cust 
df_dups = df_cust[df_cust.duplicated()]

In [19]:
#show duplicated subset
df_dups

,user_id,First Name,Last Name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income


In [20]:
#check for mixed-type data (p1,q5)

for col in df_cust.columns.tolist():
    weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_cust[weird]) > 0:
        print (col)

First Name


In [21]:
df_cust['First Name'] = df_cust['First Name'].astype('str')


# 04 Combine data

In [22]:
# look at df_opm data
df_opm.head()

,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_product_price,spending_flag,median_dspo,order_freq_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,9.0,Mid-range product,regularly busy,regularly busy time,10,new customer,6.367797,low spender,20.5,non-freq cust
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,Mid-range product,least busy,regularly busy time,10,new customer,6.367797,low spender,20.5,non-freq cust
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,Mid-range product,least busy,regularly busy time,10,new customer,6.367797,low spender,20.5,non-freq cust
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,Mid-range product,least busy,regularly busy time,10,new customer,6.367797,low spender,20.5,non-freq cust
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,Mid-range product,least busy,busiest time,10,new customer,6.367797,low spender,20.5,non-freq cust


In [23]:
# look at df_opm data types to make sure key column (user_id) will match 
df_opm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 22 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   user_order_number       int64  
 3   orders_day_of_week      int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   product_name            object 
 10  aisle_id                int64  
 11  department_id           int64  
 12  prices                  float64
 13  price_range_loc         object 
 14  busiest_days            object 
 15  busiest_period_of_day   object 
 16  max_order               int64  
 17  loyalty_flag            object 
 18  average_product_price   float64
 19  spending_flag           object 
 20  median_dspo             float64
 21  order_freq_flag         objec

In [24]:
# look at this data set to match types in df_opm dataframe (specifically user_id)
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    206209 non-null  object
 2   Last Name     206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [25]:
#look at rows/columns
df_cust.shape


(206209, 10)

In [26]:
#look at rows/columns
df_opm.shape

(32404859, 22)

In [27]:
df_ords_prods_all=df_opm.merge(df_cust, on = 'user_id', indicator = True)

In [28]:
df_ords_prods_all.head()

,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,First Name,Last Name,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [29]:
#make sure rows/columns make sense
#sum of all columns match
#same number of rows with df_opm
df_ords_prods_all.shape

(32404859, 32)

In [30]:
df_ords_prods_all['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

# Export merged dataset

In [31]:
df_ords_prods_all.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_all.pkl'))